# 📌 상품 정보 크롤링  
* urllib 라이브러리 사용해 파일 다운로드

### ✏️ 상품 페이지 목록 받아오기

In [ ]:
# 라이브러리 읽어들이기
import requests
import re
import os
from bs4 import BeautifulSoup as bs

# 파일 주소 크롤링
def bs_parsing(url):
  response = requests.get(root_url + url)
  soup = bs(response.text, "html.parser")
  elements = soup.select("table#flisttable td > a")
  
  # 주소 담을 리스트
  page_list = []
  
  # 상세 페이지 주소 리스트화
  for (idx, el) in enumerate(elements, 1):
    page_list.append(root_url + el.attrs["href"].replace("%2F", "/"))
    
  return page_list

# 리스트 값을 txt 파일에 한 줄씩 저장
def save_text_file(page_list):
  f = open("C:/Users/sodud/study/ssafy_spec_pjt/subtitle/data/page_url_list.txt", "w", encoding="UTF8")
  for page in page_list:
    f.write(page)
    f.write("\n")
  f.close
  ("[SUCCESS] 자막 페이지 목록 저장")

# 자막 사이트
root_url = "https://kitsunekko.net"

# 일본어 자막 목록 페이지
page_url = "/dirlist.php?dir=subtitles/japanese/"

# 자막 페이지 목록 리스트로 저장
page_list = bs_parsing(page_url)

# 리스트를 txt 파일로 저장
save_text_file(page_list)

### ✏️ 상품 정보 엑셀로 저장

In [78]:
import requests
import re
import os
from bs4 import BeautifulSoup
from openpyxl import Workbook

wb = Workbook(write_only=True)
ws = wb.create_sheet('상품정보')
# 컬럼 명
ws.append([ '이름', '가격', '설명 이미지', '원산지', '상품 상태', '생산자&수입자', '할인율', '리뷰 평점', '배송료', '상품사진1','상품사진2','상품사진3'])

# 임시 url
temp_url= "https://www.lotteon.com/p/product/LO1987818462?areaCode=AD_CPC&entryPoint=ad&dp_infw_cd=CASFC01200000&clickId=C33295679634"


# 웹 서버에 요청하기
# res = requests.get(temp_url)
# res.raise_for_status()


webpage = requests.get("https://www.lotteon.com/p/product/LO1987818462?areaCode=AD_CPC&entryPoint=ad&dp_infw_cd=CASFC01200000&clickId=C33295679634")
soup = BeautifulSoup(webpage.content, "html.parser")

temp = soup.select("div.productName")
temp2 = soup.select('.productName')
print(temp)
print(temp2)
temp3 = soup.find('div',class_ = 'productName')
print(temp3)

 


#     product_name = web_html.select_one('.productName').get_text()
#     upload_data = web_html.select('.top_cell__3DnEV em')[2].get_text()
#     average_grade = web_html.find(class_="top_grade__3jjdl").get_text()[2:5]
#     low_price = web_html.select('.productList_price__2FKhU em')[0].get_text()
#     high_price = web_html.select('.productList_price__2FKhU em')[-1].get_text()
#     five_point = web_html.select('.filter_top_list__3rOdK em')[1].get_text()[1:-1]
#     four_point = web_html.select('.filter_top_list__3rOdK em')[2].get_text()[1:-1]
#     three_point = web_html.select('.filter_top_list__3rOdK em')[3].get_text()[1:-1]
#     two_point = web_html.select('.filter_top_list__3rOdK em')[4].get_text()[1:-1]
#     one_point = web_html.select('.filter_top_list__3rOdK em')[5].get_text()[1:-1]
    
# print(web_html.find_all('.productName'))

# soup 객체 만들기
# soup = BeautifulSoup(temp_url, "html.parser")
# temp = soup.select("div", class_=".productName")
# print(temp)



# product_name = web_html.select_one('h2').children.get_text()
# product_price = web_html.select_one('h2').children.get_text()
# des_picture = web_html.select_one('h2').children.get_text()
# origin = web_html.select_one('h2').children.get_text()
# status = web_html.select_one('h2').children.get_text()
# producer = web_html.select_one('h2').children.get_text()
# discount_rate = web_html.select_one('h2').children.get_text()
# review_rating = web_html.select_one('h2').children.get_text()
# delivery_fee = web_html.select_one('h2').children.get_text()
# product_thumbnail_1 = web_html.select_one('h2').children.get_text()
# product_thumbnail_2 = web_html.select_one('h2').children.get_text()
# product_thumbnail_3 = web_html.select_one('h2').children.get_text()
    
    

# ws.append([product_name, product_price, des_picture, origin, status, producer, discount_rate, review_rating, delivery_fee, product_thumbnail_1, product_thumbnail_2, product_thumbnail_3 ])


# wb.save('상품정보.xlsx')

[]
[]
None


In [44]:
# 라이브러리 준비하기
import csv
import requests
from bs4 import BeautifulSoup

url ="https://comic.naver.com/webtoon/weekday"

# 엑셀 파일로 저장하기
filename = "네이버 웹툰 인기 순위.csv"
f = open(filename, "w", encoding="utf-8-sig", newline="")
writer = csv.writer(f)

columns_name = ["순위", "웹툰명"] # 컬럼 속성명 만들기

writer.writerow(columns_name)

# 웹 서버에 요청하기
res = requests.get(url)
res.raise_for_status()

# soup 객체 만들기
soup = BeautifulSoup(res.text, "lxml")
cartoonsBox = soup.find('ol', attrs={"class": "asideBoxRank"}) # 전체 영역에서 'a' 태그를 찾지 않고 인기 급상승 영역으로 범위 제한
cartoons = cartoonsBox.find_all('a') # 인기 급상승 영역에서 'a'태그 모두 찾아 변수 cartoons에 할당

i = 1

# 반복문으로 제목 가져오기(터미널 창 출력 및 엑셀 저장)
for cartoon in cartoons: 
  title = cartoon.get("title") 
  print(f"{str(i)}위: {title}")
  data = [str(i), title]
  writer.writerow(data)
  i += 1

1위: 참교육-103화
2위: 퀘스트지상주의-54화 가족 건들면 뒤진다고
3위: 윈드브레이커-4부 - 34화 마음의 평온
4위: 뷰티풀 군바리-352화_소수로서 첫 시위 (2)
5위: 신화급 귀속 아이템을 손에 넣었다-22화
6위: 팔이피플-55화 - 계모임의 향방
7위: 장씨세가 호위무사-181화
8위: 버림받은 왕녀의 은밀한 침실-18화. 한밤의 습격자
9위: 싸움독학-151화 : 보미 보고 싶다
10위: 리턴 투 플레이어-100화. 약속 (2)
